In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta

# 1. 데이터 로드 (가상 데이터 생성)
df = pd.read_csv(r'/workspace/data/CST_OP_COLLECTION.csv')
print("원본 데이터프레임의 일부:")
print(df.head())
print("\n" + "="*50 + "\n")


/tmp/ipykernel_224498/3491033604.py:7: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'/workspace/data/CST_OP_COLLECTION.csv')


원본 데이터프레임의 일부:
         EQUIP_NO          PLC_NO  REPORT_DATE  REPORT_TIME  CUST_ID  DATA1  \
0  80500336802010  175.223.31.129     20201028         1420  33680.0    136   
1  80500336802010  175.223.31.129     20201028         1425  33680.0    147   
2  80500336802010  175.223.31.129     20201028         1430  33680.0    158   
3  80500336802010  175.223.31.129     20201028         1435  33680.0    169   
4  80500336802010  175.223.31.129     20201028         1440  33680.0    180   

   DATA2    DATA3    DATA4    DATA5  ...  DATA21  DATA22  DATA23  DATA24  \
0      0  112.621  112.342  112.106  ... -14.102 -14.077    11.0     0.0   
1      0  112.621  112.342  112.106  ... -14.102 -14.077    11.0     0.0   
2      0  112.621  112.342  112.106  ... -14.102 -14.077    11.0     0.0   
3      0  112.621  112.342  112.106  ... -14.102 -14.077    11.0     0.0   
4      0  112.621  112.342  112.106  ... -14.102 -14.077    11.0     0.0   

   DATA25              CREATE_DATE               EQUI

In [2]:

# 2. 필요 컬럼 추출 및 특정 장비 번호 필터링
target_equip_no = 82501456231912
df_filtered = df[df['EQUIP_NO'] == target_equip_no].copy()
df_processed = df_filtered[['REPORT_DATE', 'REPORT_TIME', 'DATA3']].copy()
df_processed['REPORT_DATE'] = df_processed['REPORT_DATE'].astype(str)
df_processed['REPORT_TIME'] = df_processed['REPORT_TIME'].astype(str)
print(f"장비 번호 '{target_equip_no}'에 대해 필터링된 데이터 (DATA3):")
print(df_processed.head())
print("\n" + "="*50 + "\n")



장비 번호 '82501456231912'에 대해 필터링된 데이터 (DATA3):
       REPORT_DATE REPORT_TIME    DATA3
122217    20200926         705  222.989
122218    20200926         850  221.208
122219    20200926        1510  218.631
122220    20200927         205  223.145
122221    20200927         435  223.579




In [3]:

# 3. 시계열 인덱스 생성
df_processed['datetime'] = pd.to_datetime(df_processed['REPORT_DATE'] + df_processed['REPORT_TIME'], format='%Y%m%d%H%M', errors='coerce')

if df_processed['datetime'].isnull().any():
    print("경고: 'datetime' 컬럼에 유효하지 않은 날짜/시간 값이 있어 NaT로 변환되었습니다.")
    df_processed.dropna(subset=['datetime'], inplace=True)
    if df_processed.empty:
        raise ValueError("오류: 유효한 날짜/시간 데이터가 없어 처리할 데이터가 남아있지 않습니다.")

df_processed = df_processed.set_index('datetime').sort_index()

# --- 중복된 datetime 인덱스 제거 (Reindex 전에 필수) ---
# 이 부분이 오류를 해결하는 핵심입니다.
if df_processed.index.duplicated().any():
    print("경고: datetime 인덱스에 중복된 값이 있습니다. 중복된 값의 개수:", df_processed.index.duplicated().sum())
    print("중복된 인덱스 처리: 각 중복된 시간대에서 마지막 값(가장 최신 값)을 유지합니다.")
    df_processed = df_processed[~df_processed.index.duplicated(keep='last')]
else:
    print("datetime 인덱스에 중복된 값이 없습니다.")

df_processed = df_processed[['DATA3']] # DATA3 컬럼만 유지

print("datetime 인덱스로 설정 및 중복 처리된 데이터프레임:")
print(df_processed.head())
print("datetime 인덱스 NaT 값 존재 여부:", df_processed.index.isnull().any())
print("datetime 인덱스 중복 값 존재 여부 (처리 후):", df_processed.index.duplicated().any())
print("\n" + "="*50 + "\n")


경고: 'datetime' 컬럼에 유효하지 않은 날짜/시간 값이 있어 NaT로 변환되었습니다.
경고: datetime 인덱스에 중복된 값이 있습니다. 중복된 값의 개수: 320291
중복된 인덱스 처리: 각 중복된 시간대에서 마지막 값(가장 최신 값)을 유지합니다.
datetime 인덱스로 설정 및 중복 처리된 데이터프레임:
                       DATA3
datetime                    
2020-01-10 01:00:00  224.224
2020-01-10 01:05:00  224.362
2020-01-10 02:00:00  223.058
2020-01-10 02:05:00  223.770
2020-09-02 06:05:00  221.963
datetime 인덱스 NaT 값 존재 여부: False
datetime 인덱스 중복 값 존재 여부 (처리 후): False




In [4]:

# 4. 누락된 시간 처리 및 결측치 보간
# 시계열의 시작과 끝을 기준으로 5분 간격의 모든 시간을 포함하는 datetime index 생성
start_date = df_processed.index.min()
end_date = df_processed.index.max()
full_time_index = pd.date_range(start=start_date, end=end_date, freq='5min')

# 원본 데이터프레임을 전체 시간 인덱스에 재색인 (Reindex)
# 이때, 존재하지 않는 시간은 NaN으로 채워짐
df_reindexed = df_processed.reindex(full_time_index)

# NaN (결측치) 처리: 선형 보간
# 시계열 데이터의 경우, 이전/이후 값의 경향성을 따라 채우는 것이 일반적
df_reindexed['DATA3'] = df_reindexed['DATA3'].interpolate(method='time')

# 혹시 맨 앞이나 맨 뒤에 NaN이 있어서 interpolate로 채워지지 않는 경우 (거의 없겠지만)
# backfill이나 ffill로 한 번 더 처리할 수 있습니다.
df_reindexed['DATA3'] = df_reindexed['DATA3'].fillna(method='bfill')
df_reindexed['DATA3'] = df_reindexed['DATA3'].fillna(method='ffill')


print("시간 재색인 및 결측치 보간 후 데이터프레임 (일부러 NaN을 넣었다면 확인 가능):")
print(df_reindexed.head(10)) # 더 많은 행을 출력하여 보간 확인
print(f"총 데이터 포인트 수: {len(df_reindexed)}")
print(f"결측치 확인 (NaN 개수): {df_reindexed['DATA3'].isnull().sum()}")
print("\n" + "="*50 + "\n")

# 5. 정규화 (Min-Max Scaling)
# LSTM은 입력 값의 스케일에 민감하므로 정규화는 필수적입니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_reindexed['DATA3'].values.reshape(-1, 1))

# 스케일링된 데이터를 DataFrame으로 다시 변환
df_scaled = pd.DataFrame(scaled_data, index=df_reindexed.index, columns=['DATA3_scaled'])

print("Min-Max Scaling 후 데이터프레임 (0과 1 사이 값):")
print(df_scaled.head())
print(f"최소값: {df_scaled['DATA3_scaled'].min()}, 최대값: {df_scaled['DATA3_scaled'].max()}")
print("\n" + "="*50 + "\n")

# 6. 시퀀스 데이터 생성 함수 (LSTM 입력 형태)
def create_sequences(data, look_back):
    """
    LSTM 모델 학습을 위한 시퀀스 데이터셋을 생성합니다.

    Args:
        data (numpy.array): 정규화된 시계열 데이터 (단일 특성).
        look_back (int): 과거 몇 개의 타임스텝을 보고 미래를 예측할지 (시퀀스 길이).

    Returns:
        tuple: (X, y) 형태의 NumPy 배열.
               X는 LSTM 입력 시퀀스, y는 예측 대상 값.
    """
    X, y = [], []
    for i in range(len(data) - look_back):
        seq_in = data[i:(i + look_back), 0]
        seq_out = data[i + look_back, 0]
        X.append(seq_in)
        y.append(seq_out)
    return np.array(X), np.array(y)

# look_back (윈도우 크기) 설정 예시
# 예를 들어, 과거 12개 (1시간 분량, 5분 간격)의 데이터를 보고 다음 5분 뒤를 예측
look_back = 12
X, y = create_sequences(scaled_data, look_back)

# LSTM 입력 형태에 맞게 X 데이터 reshape: [샘플 수, 타임스텝, 특징 수]
# 현재는 단일 특징 (DATA3)이므로 특징 수는 1입니다.
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print(f"생성된 시퀀스 데이터 (X) 형태: {X.shape} (샘플 수, 윈도우 크기, 특징 수)")
print(f"생성된 예측 대상 데이터 (y) 형태: {y.shape} (샘플 수)")
print(f"X의 첫 번째 샘플:\n{X[0].flatten()}")
print(f"y의 첫 번째 값: {y[0]}")
print("\n" + "="*50 + "\n")

print("데이터 준비 및 전처리 완료.")
print("생성된 X와 y는 이제 LSTM 모델 학습에 바로 사용될 수 있습니다.")
print("스케일러 (scaler) 객체는 나중에 예측값을 다시 원본 스케일로 되돌릴 때 사용됩니다.")

시간 재색인 및 결측치 보간 후 데이터프레임 (일부러 NaN을 넣었다면 확인 가능):
                          DATA3
2020-01-10 01:00:00  224.224000
2020-01-10 01:05:00  224.362000
2020-01-10 01:10:00  224.243455
2020-01-10 01:15:00  224.124909
2020-01-10 01:20:00  224.006364
2020-01-10 01:25:00  223.887818
2020-01-10 01:30:00  223.769273
2020-01-10 01:35:00  223.650727
2020-01-10 01:40:00  223.532182
2020-01-10 01:45:00  223.413636
총 데이터 포인트 수: 85802
결측치 확인 (NaN 개수): 0


Min-Max Scaling 후 데이터프레임 (0과 1 사이 값):
                     DATA3_scaled
2020-01-10 01:00:00      0.874376
2020-01-10 01:05:00      0.885137
2020-01-10 01:10:00      0.875893
2020-01-10 01:15:00      0.866649
2020-01-10 01:20:00      0.857405
최소값: 0.0, 최대값: 1.0


생성된 시퀀스 데이터 (X) 형태: (85790, 12, 1) (샘플 수, 윈도우 크기, 특징 수)
생성된 예측 대상 데이터 (y) 형태: (85790,) (샘플 수)
X의 첫 번째 샘플:
[0.87437617 0.88513724 0.87589321 0.86664918 0.85740515 0.84816112
 0.83891709 0.82967306 0.82042903 0.81118499 0.80194096 0.79269693]
y의 첫 번째 값: 0.7834529008109783


데이터 준비 및 전처리 완료.
생성된 X와 

/tmp/ipykernel_224498/3024218888.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_reindexed['DATA3'] = df_reindexed['DATA3'].fillna(method='bfill')
/tmp/ipykernel_224498/3024218888.py:18: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_reindexed['DATA3'] = df_reindexed['DATA3'].fillna(method='ffill')


In [6]:
print("df_processed 인덱스 확인:")
print(df_processed.index)
print("df_processed 인덱스에 NaT 값 존재 여부:", df_processed.index.isnull().any())

df_processed 인덱스 확인:
DatetimeIndex([], dtype='datetime64[ns]', name='datetime', freq=None)
df_processed 인덱스에 NaT 값 존재 여부: False
